In [5]:
import numpy as np

class SimpleGCN:
    def __init__(self, in_features, hidden_features, out_features, lr=0.01, seed=0):
        np.random.seed(seed)
        # Xavier initialization
        limit1 = np.sqrt(6 / (in_features + hidden_features))
        limit2 = np.sqrt(6 / (hidden_features + out_features))
        
        self.W1 = np.random.uniform(-limit1, limit1, (in_features, hidden_features))
        self.W2 = np.random.uniform(-limit2, limit2, (hidden_features, out_features))
        self.lr = lr

    def relu(self, x):
        return np.maximum(0, x)

    def relu_grad(self, x):
        return (x > 0).astype(float)

    def softmax(self, x):
        exp_x = np.exp(x - np.max(x, axis=1, keepdims=True))
        return exp_x / np.sum(exp_x, axis=1, keepdims=True)

    def forward(self, A_norm, X):
        """ Forward pass with cached intermediates for backprop """
        self.X = X
        self.A_norm = A_norm

        self.H_pre = A_norm @ X @ self.W1     # before ReLU
        self.H = self.relu(self.H_pre)        # hidden layer
        self.Z = A_norm @ self.H @ self.W2    # logits
        self.out = self.softmax(self.Z)       # probabilities
        return self.out

    def compute_loss(self, Y_true):
        """ Cross-entropy loss """
        m = Y_true.shape[0]
        log_likelihood = -np.log(self.out[range(m), Y_true] + 1e-9)
        return np.sum(log_likelihood) / m

    def backward(self, Y_true):
        """ Backpropagation for W1 and W2 """
        m = Y_true.shape[0]
        Y_onehot = np.zeros_like(self.out)
        Y_onehot[np.arange(m), Y_true] = 1

        # dL/dZ
        dZ = (self.out - Y_onehot) / m

        # Gradients for W2
        dW2 = self.H.T @ (self.A_norm.T @ dZ)

        # Backprop to hidden
        dH = (self.A_norm @ dZ) @ self.W2.T
        dH_pre = dH * self.relu_grad(self.H_pre)

        # Gradients for W1
        dW1 = self.X.T @ (self.A_norm.T @ dH_pre)

        # Update weights
        self.W1 -= self.lr * dW1
        self.W2 -= self.lr * dW2

    def train(self, A, X, Y, epochs=200):
        # Precompute normalized adjacency
        I = np.eye(A.shape[0])
        A_hat = A + I
        D_hat = np.diag(1.0 / np.sqrt(np.sum(A_hat, axis=1)))
        A_norm = D_hat @ A_hat @ D_hat

        for epoch in range(epochs):
            out = self.forward(A_norm, X)
            loss = self.compute_loss(Y)
            self.backward(Y)
            if epoch % 200 == 0 or epoch == epochs-1:
                preds = np.argmax(out, axis=1)
                acc = np.mean(preds == Y)
                print(f"Epoch {epoch:03d}: loss={loss:.4f}, acc={acc:.4f}")

# -------------------------------


In [6]:
# Example usage
if __name__ == "__main__":
    # Example graph (4 nodes)
    A = np.array([
        [0, 1, 0, 0],
        [1, 0, 1, 1],
        [0, 1, 0, 1],
        [0, 1, 1, 0]
    ], dtype=float)

    # Node features: 4 nodes × 3 features
    X = np.random.randn(4, 3)

    # Node labels (classes 0 or 1)
    Y = np.array([0, 1, 0, 1])

    gcn = SimpleGCN(in_features=3, hidden_features=5, out_features=2, lr=0.1)
    gcn.train(A, X, Y, epochs=10000)


Epoch 000: loss=0.7209, acc=0.5000
Epoch 200: loss=0.6689, acc=0.7500
Epoch 400: loss=0.6621, acc=0.7500
Epoch 600: loss=0.6564, acc=0.7500
Epoch 800: loss=0.6518, acc=0.7500
Epoch 1000: loss=0.6471, acc=0.7500
Epoch 1200: loss=0.6409, acc=0.7500
Epoch 1400: loss=0.6336, acc=0.7500
Epoch 1600: loss=0.6266, acc=0.7500
Epoch 1800: loss=0.6184, acc=0.7500
Epoch 2000: loss=0.6077, acc=0.7500
Epoch 2200: loss=0.5945, acc=0.7500
Epoch 2400: loss=0.5793, acc=0.7500
Epoch 2600: loss=0.5632, acc=0.7500
Epoch 2800: loss=0.5475, acc=0.7500
Epoch 3000: loss=0.5335, acc=0.7500
Epoch 3200: loss=0.5219, acc=0.7500
Epoch 3400: loss=0.5126, acc=0.7500
Epoch 3600: loss=0.5062, acc=0.7500
Epoch 3800: loss=0.5022, acc=0.7500
Epoch 4000: loss=0.4993, acc=0.7500
Epoch 4200: loss=0.4971, acc=0.7500
Epoch 4400: loss=0.4954, acc=0.7500
Epoch 4600: loss=0.4940, acc=0.7500
Epoch 4800: loss=0.4930, acc=0.7500
Epoch 5000: loss=0.4921, acc=0.7500
Epoch 5200: loss=0.4914, acc=0.7500
Epoch 5400: loss=0.4908, acc=0.75

In [8]:
gcn.A_norm

array([[0.5       , 0.35355339, 0.        , 0.        ],
       [0.35355339, 0.25      , 0.28867513, 0.28867513],
       [0.        , 0.28867513, 0.33333333, 0.33333333],
       [0.        , 0.28867513, 0.33333333, 0.33333333]])